# Análise de Dados Exploratória da ANAC

![ANAC](imgs/anac-logo.png "Logo ANAC")

# 1. Introdução

Este estudo tem como objetivo analisar um conjunto de dados de voos internacionais registrados pela Agência Nacional de Aviação Civil (ANAC) do Brasil, durante o período de janeiro de 2015 a agosto de 2017. 

A ANAC é a agência reguladora responsável pela aviação civil no país, e as informações contidas nesses dados são de grande importância para a compreensão da dinâmica da aviação internacional no Brasil, bem como para a tomada de decisões estratégicas no setor. O conjunto de dados inclui informações como o número do voo, nome da companhia aérea, horários de partida e chegada previstos e reais, situação do voo e localização dos aeroportos de origem e destino, entre outros. Esses dados permitem uma análise detalhada das tendências e padrões de voos internacionais no Brasil, bem como a identificação de fatores que possam influenciar o desempenho da aviação internacional no país.

## 1.1. Objetivos

O objetivo deste estudo é realizar uma Análise Exploratória dos Dados (Exploratory Data Analysis - EDA) do conjunto de dados disposto em [bases/](bases/), e analisar dados de voos internacionais rastreados pela ANAC.

Com base nas informações disponíveis, pretendemos avaliar o desempenho das companhias aéreas em relação ao cumprimento dos horários previstos de partida e chegada, bem como investigar a frequência de cancelamentos e as justificativas apresentadas para esses casos. 

Também vamos examinar as rotas mais utilizadas e os destinos mais comuns para os voos internacionais saindo do Brasil, bem como avaliar a pontualidade e a eficiência dos aeroportos de origem e destino. 

1. [Análise 1](#Análise1)
2. [Análise 2](#Análise2)
3. [Análise 3](#Análise3)
4. [Análise 4](#Análise4)


# 2. Metodologia

Nesta seção será apresentado todo o processo de preparação, organização e limpeza de dados feito no *dataset* que possui os seguintes dados:

| Coluna               | Tipo     | Descrição                                                 |
|----------------------|----------|-----------------------------------------------------------|
| voos                 | str      | Número do voo                                             |
| companhia_aerea      | str      | Nome da companhia aérea                                   |
| codigo_tipo_linha    | str      | Código do tipo de linha (internacional)                   |
| partida_prevista     | datetime | Horário previsto de partida                               |
| partida_real         | datetime | Horário real de partida                                   |
| chegada_prevista     | datetime | Horário previsto de chegada                               |
| chegada_real         | datetime | Horário real de chegada                                   |
| situacao_voo         | str      | Situação do voo (realizado, cancelado, etc)               |
| codigo_justificativa | str      | Código de justificativa (caso o voo tenha sido cancelado) |
| aeroporto_origem     | str      | Código do aeroporto de origem                             |
| cidade_origem        | str      | Cidade de origem                                          |
| estado_origem        | str      | Estado de origem                                          |
| pais_origem          | str      | País de origem                                            |
| aeroporto_destino    | str      | Código do aeroporto de destino                            |
| cidade_destino       | str      | Cidade de destino                                         |
| estado_destino       | str      | Estado de destino                                         |
| pais_destino         | str      | País de destino                                           |
| long_dest             | int      | Longitude do aeroporto de destino                         |
| lat_dest              | int      | Latitude do aeroporto de destino                          |
| long_orig             | int      | Longitude do aeroporto de origem                          |
| lat_orig              | int      | Latitude do aeroporto de origem                           |

## 2.1. Configuração do Ambiente

### 2.1.1. Bibliotecas e Ferramentas Necessárias (Built-ins)

1. ```os```:       Para manipular os arquivos de base e seus respectivos diretórios de forma multiplataforma e eficiente.
2. ```re```:       Utilizado em expressões regulares durante o processo de limpeza dos dados.
3. ```enum```:     Auxilia nos processos de padronizar e regular algumas convenções do projeto.
4. ```datetime```: Auxilia nos processos de padronizar e regular algumas convenções do projeto

### 2.1.2. Bibliotecas e Ferramentas Necessárias (pip)

1. Pandas: Para manipular e exibir os dados de forma coesa e relata.

    ```pip install pandas```       
    ```import pandas as pd```

2. JinJa2: Complemento de estilização do Pandas.

    ```pip install Jinja2```

3. DescricaoColuna e DescricaoTipo: São classes estáticas compostar por Enums estabelecidos com base nas convenções de nome das colunas do *dataset*.   
   **Utils** possui complementos para tratativas de alguns campos.   
   **DescricaoColuna** é responsável pelas informações de descrição dos campos.   
   **DescricaoTipo** é responsável pelas informações de tipo dos campos.

    ```from libs.utils import Utils, DescricaoColuna, DescricaoTipo```

### 2.1.3. Importações gerais para o projeto

In [1]:
import os
import re
import pandas as pd

from libs.utils import Utils, DescricaoColuna, DescricaoTipo

## 2.2. Carregamento da base

In [2]:
caminho = os.path.join(
            os.path.dirname(
                os.path.abspath(
                    os.path.join(__name__, '.'))), 'bases')

arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith(".csv")]

dataframes = [pd.read_csv(os.path.join(caminho, arquivo)) for arquivo in arquivos]

flights_dataframe = pd.concat(dataframes)

## 2.3. Organização e Limpeza dos Dados

### 2.3.1. Padrozinação dos nomes das colunas

In [3]:
nomes_colunas_tratadas = []

for coluna in flights_dataframe.columns:
    if any(char.isupper() for char in coluna):
        nomes_colunas_tratadas.append('_'.join(word.lower() for word in re.findall('[A-Z][^A-Z]*', coluna)).lower().replace('.', ''))

flights_dataframe.columns = nomes_colunas_tratadas

### 2.3.2. Padrozinação dos tipos das colunas de data/hora

In [4]:
for coluna in flights_dataframe.columns:
    tipo = DescricaoTipo.get(coluna)
    if tipo.__name__ == 'datetime': 
        flights_dataframe[coluna] = Utils.formatar_data_hora(flights_dataframe, coluna)

### 2.3.3. Gerando coluna que contém a distância em KM (para *lat_long_orig* e *lat_long_dest*)

In [5]:
flights_dataframe['distancia_km'] = Utils.calcular_distancia_km(
    flights_dataframe['lat_orig'],
    flights_dataframe['long_orig'],
    flights_dataframe['lat_dest'],
    flights_dataframe['long_dest']
)

### 2.3.1. Removendo 'N/I' por um valor ''/string.empty

In [6]:
for coluna in flights_dataframe.columns:
    flights_dataframe[coluna] = flights_dataframe[coluna].replace('N/I', '')

### 2.3.5. Removendo np.NaN por um valor None

In [7]:
flights_dataframe.fillna(value='', inplace=True)

### 2.3.n Resultado final

In [8]:
columns = ['NomeColuna', DescricaoColuna.__name__, DescricaoTipo.__name__]
values = [[value, DescricaoColuna.get(value), DescricaoTipo.get(value).__name__] for value in flights_dataframe.columns]
propertys = [dict(selector='th', props=[('text-align', 'center')]), dict(selector='td', props=[('text-align', 'center')])]
pd.DataFrame(data=values, columns=columns).style.set_table_styles(propertys)

,NomeColuna,DescricaoColuna,DescricaoTipo
0,voos,Número do voo,str
1,companhia_aerea,Nome da companhia aérea,str
2,codigo_tipo_linha,Código do tipo de linha (internacional),str
3,partida_prevista,Horário previsto de partida,datetime
4,partida_real,Horário real de partida,datetime
5,chegada_prevista,Horário previsto de chegada,datetime
6,chegada_real,Horário real de chegada,datetime
7,situacao_voo,"Situação do voo (realizado, cancelado, etc)",str
8,codigo_justificativa,Código de justificativa (caso o voo tenha sido cancelado),str
9,aeroporto_origem,Código do aeroporto de origem,str


# 3. Análise dos Dados

## 3.1. Quantidade de voos por companhia aérea.
Verificar a quantidade de voos realizados por cada companhia aérea, ou seja, analisar a distribuição de frequência da coluna 'Companhia.Aerea'.

## 3.2. Quantidade de voos por tipo de linha.
Verificar a quantidade de voos internacionais, nacionais e regionais, ou seja, analisar a distribuição de frequência da coluna 'Codigo.Tipo.Linha'.

## 3.3. Pontualidade dos voos.
Analisar a pontualidade dos voos, comparando a hora de partida prevista com a hora de partida real e a hora de chegada prevista com a hora de chegada real, utilizando gráficos de dispersão ou histogramas, e identificar os voos que tiveram maior atraso.

### 3.3.1 Pontualidade por companhia aérea.


### 3.3.2 Pontualidade por tipo de linha.


## 3.4. frequência de voos cancelados.
Investigar a frequência de voos cancelados em função da justificativa apresentada na coluna 'Codigo.Justificativa'.

## 3.5. Correlação entre a localização dos aeroportos de origem e destino.
Analisar a correlação entre a localização dos aeroportos de origem e destino (latitude e longitude) e as companhias aéreas e tipo de linha dos voos utilizando gráficos de dispersão.

## 3.6. Aeroportos mais movimentados.
Identificar os aeroportos com maior quantidade de voos de partida e de chegada, utilizando gráficos de barras.

## 3.7. Distribuição geográfica dos voos internacionais
Analisar a distribuição geográfica dos voos internacionais em função dos países de origem e destino, utilizando mapas.